In [1]:
import pandas as pd
import numpy as np

## My Package

In [10]:
# DISTRIBUTIONS (NORMAL AND T)
from risk_management import distributions

# mu_vector, covariance_matrix = fit_multivariate_normal_dist(x: pd.DataFrame)

# mu, sigma, nu = fit_univariate_t_dist(x)

# alpha, betas, mu, sigma, nu = t_regression(X: pd.DataFrame, y:pd.Series, add_constant:bool = True, print_summary = False)

In [12]:
# MEASUREMENTS (CORRELATION, COVARIANCE, FACTORIZATIONS, RETURNS)
from risk_management import measurements

# corr = compute_correlation(x:pd.DataFrame, method="pearson", drop_missing = False, exponentially_weighted = False, lambda_ = 0.97, ddof: int =1)

# cov = compute_covariance(x:pd.DataFrame, drop_missing = False, exponentially_weighted = False, lambda_ = 0.97, ddof:int=1)

# cov = compute_covariance_with_ew_corr(x: pd.DataFrame, corr_lambda, var_lambda)

# psd = near_psd(A: pd.DataFrame, epsilon = 0.0)
# psd = higham_psd(A: pd.DataFrame, tolerance = 1e-8, max_iterations= 100_000)

# A = cholesky_factor(x:pd.DataFrame)

# returns = compute_returns(x: pd.DataFrame, return_type = "arithmetic")


In [13]:
# RISK METRICS (VAR, ES)
from risk_management import risk_metrics

# NOTE -> all of these assume that X was a set of RETURNS, not prices

# abs_VaR, rel_VaR = univariate_normal_VaR(mean: float, std: float, alpha = 0.05)

# nu, mu, sigma = t.fit(x)
# abs_VaR, rel_VaR = univariate_t_VaR(mu: float, sigma: float, nu: float, alpha: float = 0.05)

# abs_ES, diff_ES = expected_shortfall_normal(x:pd.Dataframe, mu:float, sigma: float, alpha = 0.05)

# mu, sigma, nu = fit_univariate_t_dist(x)
# abs_ES, diff_ES = expected_shortfall_t(mu: float, sigma: float, nu: float, alpha: float = 0.05)



In [15]:
## SIMULATIONS
from risk_management import simulations

# NOTE -> all of these assume that X was a set of RETURNS, not prices

# simulation_data = normal_monte_carlo_simulation(mean_vector, covariance_matrix, n_sims, fix_method, seed=1234) # len(cov), n_sims

# simulation_data = pca_monte_carlo_simulation(mean_vector, covariance_matrix, n_sims, explained_threshold = 0.99, seed=1234) # len(cov), n_sims

# current_prices = vector of start prices of assets, holdings = vector of how many of each asset we have
# abs_VaR, rel_VaR = monte_carlo_VaR_sim(mean_vector, covariance_matrix, current_prices, holdings, n_draws, return_type = "arithmetic", alpha = 0.05, seed = 1234) 

# sim_dataframe = VaR_ES_2_level_sim_from_copula(sample_data: pd.DataFrame, holdings: np.array, prices: np.array, fix_method, n_sims = 100_000, alpha=0.05, seed=1234)

## Problem 1

## Problem 2

In [2]:
data_2 = pd.read_csv("problem2.csv")

In [16]:
data_2

,X
0,0.009238
1,-0.043426
2,-0.017099
3,0.050770
4,0.018022
...,...
995,0.066770
996,0.002401
997,0.005536
998,-0.031395


## Problem 3

## Problem 4

In [3]:
data_4 = pd.read_csv("problem4.csv")

In [17]:
data_4

,x1,x2,x3
0,0.034246,0.121262,-0.115528
1,0.131169,0.192027,0.043622
2,0.125515,-0.108881,-0.004787
3,0.157331,0.033694,0.109034
4,-0.119847,-0.249491,-0.234808
...,...,...,...
495,-0.087194,-0.062952,-0.125135
496,-0.015536,-0.134150,0.012078
497,-0.182251,-0.144493,-0.240579
498,0.167292,0.207168,0.128010


## Problem 5

In [4]:
data_5 = pd.read_csv("problem5.csv")

In [18]:
data_5

,x1,x2,x3,x4,x5
0,0.378172,0.508313,NaN,0.610538,NaN
1,0.858478,0.758142,NaN,NaN,NaN
2,-0.218954,-0.015334,-0.261880,NaN,-0.230148
3,-0.370442,-0.424702,-0.535312,NaN,-0.226233
4,0.717732,0.664195,NaN,NaN,0.914107
5,1.331701,NaN,1.464934,1.156675,1.397827
6,1.462355,NaN,1.396949,1.476854,NaN
7,0.560968,0.570253,0.622116,0.570309,0.417396
8,NaN,1.343589,1.578797,1.576128,NaN
9,NaN,NaN,-2.841926,-2.845942,NaN


## Problem 6

In [23]:
data_6 = pd.read_csv("problem6.csv")
# c = data_6.set_index("Date")

In [24]:
data_6


,x1,x2,x3,Date
0,102.975983,101.234230,101.390965,2025-01-01
1,102.553769,102.056711,103.589863,2025-01-02
2,102.362551,100.605815,99.204933,2025-01-03
3,104.522707,100.479907,101.798573,2025-01-04
4,103.682209,100.637430,101.599604,2025-01-05
...,...,...,...,...
245,79.313328,74.647039,78.285858,2025-09-03
246,78.871520,74.591839,77.059249,2025-09-04
247,80.080646,74.965529,77.739804,2025-09-05
248,81.411350,75.702936,79.729444,2025-09-06
